<a href="https://colab.research.google.com/github/ppokranguser/Artificial_Intelligence_study/blob/main/240924_logistic_regression_blanks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Logistic Regression using PyTorch!

# Import Packages

In [ ]:
# import packeges here

# Make the dataset!

In [ ]:
# Make the dataset here


# Define the model

In [ ]:
# Create model


# Define a loss function and training process

In [ ]:
# Loss and optimizer


# training loop



# Plot the result : training loss

In [ ]:
# plot the results


# Evaluate the trained model using the testset

In [ ]:
# evaluate the trained model
